In [ ]:
from keras.models import load_model # type: ignore
from keras.applications.imagenet_utils import preprocess_input # type: ignore

import cv2
import numpy as np
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [3]:
class_names = [
    "Speed limit 20", "Speed limit 30", "Speed limit 50", "Speed limit 60",
    "Speed limit 70", "Speed limit 80", "End of speed limit 80", "Speed limit 100",
    "Speed limit 120", "No passing", "No passing for trucks", "Right-of-way at intersection",
    "Priority road", "Yield", "Stop", "No vehicles", "No trucks",
    "No entry", "General caution", "Dangerous curve left", "Dangerous curve right",
    "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right",
    "Road work", "Traffic signals", "Pedestrians", "Children crossing",
    "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing",
    "End of all restrictions", "Turn right ahead", "Turn left ahead",
    "Ahead only", "Go straight or right", "Go straight or left", "Keep right",
    "Keep left", "Roundabout", "End of no passing", "End of no passing for trucks"
]

def preprocess_sign(image):
    image = cv2.resize(image, (32, 32))            # Resize to model input size
    image = image.astype("float32") / 255.0         # Normalize
    image = np.expand_dims(image, axis=0)           # Add batch dimension
    return image


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO

yolo_model = YOLO('yolov8s.pt')  

cnn_model = load_model("cnn_model_v02.h5")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = yolo_model(frame)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        class_id = int(box.cls[0])
        conf = float(box.conf[0])

        cropped_img = frame[y1:y2, x1:x2]

        if cropped_img.size == 0:
            continue

        processed_img = preprocess_sign(cropped_img)
        prediction = cnn_model.predict(processed_img)

        label = class_names[np.argmax(prediction)]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("YOLOv8 + CNN", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
yolo_model = YOLO("yolov8s.pt")
cnn_model = load_model('cnn_model_v01.h5')


image_folder = "GTSDB/images/train"
output_file = "yolo_cnn_predictions.txt"


with open(output_file, "w") as f:
    for img_name in os.listdir(image_folder):
        if img_name.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(image_folder, img_name)
            image = cv2.imread(img_path)

            results = yolo_model.predict(img_path, save=False, conf=0.25, verbose=False)
            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)  # (x1, y1, x2, y2)

            f.write(f"{img_name} ")

            for box in boxes:
                x1, y1, x2, y2 = box
                crop = image[y1:y2, x1:x2]

                pred = cnn_model.predict(preprocess_sign(crop), verbose=0)
                class_id = np.argmax(pred)

                f.write(f"{class_id} ")

            f.write("\n")

In [13]:
with open("yolo_cnn_predictions.txt", "r") as f:
    lines = f.readlines()

In [14]:
pred = []
for line in lines:
    parts = line.strip().split(' ')
    img_name = parts[0]
    if len(parts) > 1:
        class_ids = [int(cls_id) for cls_id in parts[1:]]
    else:
        class_ids = [99]
    if class_ids:
        pred.append((img_name, class_ids))

In [15]:
actual = []
for file in os.listdir("GTSDB/labels/train"):
    with open(os.path.join("GTSDB/labels/train", file), "r") as f:
        content = f.readlines()
        content = [line.strip() for line in content if line.strip()]
        if content:
            class_id = []
            for line in content:
                class_id.append(int(line.split()[0]))
            actual.append((file, class_id))   

In [16]:
sum = 0
count = 0

for i in range(len(actual)):
    count += (max(len(actual[i][1]), len(pred[i][1])))

for i in range(len(actual)):
    for j in pred[i][1]:
        if j in actual[i][1]:
            sum += 1
accuracy = sum / count
accuracy

0.05016181229773463